source: https://github.com/magenta/magenta/tree/main/magenta/models/music_vae
https://www.rootstrap.com/blog/how-to-generate-music-with-ai/

# Making music with Magenta

[Magenta](https://magenta.tensorflow.org/) is a Python library that helps you generate art and music. In this tutorial, we'll talk about the music generation bits in `note_seq` -- how to make your browser sing, and in particular, how to make your browser sing like you!

As a library, `note_seq` can help you:
- make music using some of the neat abstractions and utilities in the library
- use Machine Learning models to generate music.

# Basic Instructions

1. Double click on the hidden cells to make them visible, or select "View > Expand Sections" in the menu at the top.
2. Hover over the "`[ ]`" in the top-left corner of each cell and click on the "Play" button to run it, in order.
3. Listen to the generated samples.
4. Make it your own: copy the notebook, modify the code, train your own models, upload your own MIDI, etc. See the Magenta code on [GitHub](https://github.com/magenta/magenta) for more information!

# Step 0: First things first!
If you're going to use `Magenta`, you need to install it and its dependencies. Some of the later examples will also download other dependencies (such as models and checkpoints)

In [1]:
#@test {"output": "ignore"}
print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print('Importing libraries and defining some helper functions...')
from google.colab import files

import magenta
import note_seq
import tensorflow

print('🎉 Done!')
print(magenta.__version__)
print(tensorflow.__version__)

Installing dependencies...
Importing libraries and defining some helper functions...


/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.8/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB thr

🎉 Done!
2.1.4
2.9.1


## Useful helpers
There are a lot of other helper methods sprinkled around the `note_seq` codebase that you might need but not know where to find. Here are some of our favourites:

- [converting](https://github.com/magenta/note-seq/blob/main/note_seq/midi_io.py) between MIDI and NoteSequences
- [trimming, concatenating and expanding](https://github.com/magenta/note-seq/blob/main/note_seq/sequences_lib.py) NoteSequences
- [colab notebook](https://github.com/magenta/note-seq/blob/main/note_seq/notebook_utils.py) utils

# Step 2. Using Machine Learning to make music

`note_seq` has several Machine Learning models, each with different strengths. All models are built with [Tensorflow](https://www.tensorflow.org), so they will run faster if you can run them on a GPU. Here are some of the most popular ones:

- [**MelodyRNN**](https://github.com/magenta/magenta/tree/main/magenta/models/melody_rnn) - you give it a NoteSequence, and it continues it in the style of your original NoteSequence.
- [**MusicVAE**](https://github.com/magenta/magenta/tree/main/magenta/models/music_vae) - generates brand new NoteSequences or interpolates between two sequences.
- [**Onsets and Frames**](https://github.com/magenta/magenta/tree/main/magenta/models/onsets_frames_transcription) -- transcribes piano audio

Now that we know how to use `NoteSequences`, adding some basic Machine Learning is a continuation of that. The pattern for using any of these models is:

- Load `note_seq` (which we already know how to do!)
- Create a model from a downloaded checkpoint (i.e. where the weights, or the encoding, of the model lives)
- Ask the model to do something.

## Music VAE

A [MusicVAE](https://g.co/magenta/musicvae) is a variational autoencoder made up of an Encoder and Decoder -- you can think of the encoder as trying to summarize all the data you give it, and the decoder as trying to recreate the original data, based on this summarized version. As a generative model, you can think of a VAE as coming up with new sequences that could be a decoding of some summarized version.

The Music VAE implementation in `magenta/music` in particular does two things: it can create new sequences (which are reconstructions or variations of the input data), or it can interpolate between two.

### Initialize the model

In [3]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


Convert our input files into note_sequences. NOTE: I downloaded the sample lofi songs onto my drive and used that as my input directory. Change as necessary

In [4]:
!convert_dir_to_note_sequences \
  --input_dir=/content/gdrive/MyDrive/samples \ 
  --output_file=/content/gdrive/MyDrive/samples/tmp/notesequences.tfrecord \ 
  --recursive

/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.8/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB thr

Train VAE network. Choose any config you would like - this will effect the conditioning of our network. config examples here https://github.com/magenta/magenta/blob/main/magenta/models/music_vae/configs.py

In [11]:
!music_vae_train \
--config=hier-multiperf_vel_1bar_med \
--run_dir=/content/gdrive/MyDrive/samples/tmp/ \
--mode=train \
--examples_path=/content/gdrive/MyDrive/samples/tmp/notesequences.tfrecord

/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.8/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB thr

In [12]:
# print('Copying checkpoint from GCS. This will take less than a minute...')
# # This will download the mel_2bar_big checkpoint. There are more checkpoints that you
# # can use with this model, depending on what kind of output you want
# # See the list of checkpoints: https://github.com/magenta/magenta/tree/main/magenta/models/music_vae#pre-trained-checkpoints
# !gsutil -q -m cp -R gs://download.magenta.tensorflow.org/models/music_vae/colab2/checkpoints/mel_2bar_big.ckpt.* /content/

# Import dependencies.
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

# Initialize the model.
print("Initializing Music VAE...")
music_vae = TrainedModel(
      configs.CONFIG_MAP['hier-multiperf_vel_1bar_med'], 
      batch_size=4, 
      checkpoint_dir_or_path='/content/gdrive/MyDrive/samples/tmp/train')

print('🎉 Done!')


Initializing Music VAE...
🎉 Done!


### Creating new sequences

With Music VAE, you can configure how many new sequences to generate, the number of steps the new sequence will be, as well as the "temperature" of the result -- the higher the temperature, the more random (and less like the input) your sequence will be. You can play around with these values and see how the resulting sequences are different:

In [29]:
generated_sequences = music_vae.sample(n=10, length=100, temperature=1.2)

total = note_seq.concatenate_sequences(generated_sequences)

note_seq.plot_sequence(total)
note_seq.play_sequence(total, synth=note_seq.fluidsynth)

In [27]:
 note_seq.note_sequence_to_midi_file(total, "/content/gdrive/MyDrive/samples/tmp/generated/output.mid")

### Interpolating between two sequences

In [ ]:
# We're going to interpolate between the Twinkle Twinkle Little Star
# NoteSequence we defined in the first section, and one of the generated
# sequences from the previous VAE example

# How many sequences, including the start and end ones, to generate.
num_steps = 8;

# This gives us a list of sequences.
note_sequences = music_vae.interpolate(
      twinkle_twinkle,
      teapot, 
      num_steps=num_steps,
      length=32)

# Concatenate them into one long sequence, with the start and 
# end sequences at each end. 
interp_seq = note_seq.sequences_lib.concatenate_sequences(note_sequences)

note_seq.play_sequence(interp_seq, synth=note_seq.fluidsynth)
note_seq.plot_sequence(interp_seq)

NameError: ignored

# That's it!

You're now ready to build your own amazing, Machine Learning powered, music instrument! If you want more information, you can check out:

- some [demos](https://magenta.tensorflow.org/demos) `#MadeWithMagenta`
- some more of our [Colab notebooks](https://magenta.tensorflow.org/demos#colab-notebooks)
- the [documentation](https://github.com/magenta/magenta)
- the [Magenta blog](https://magenta.tensorflow.org/blog), which talks about all the mathy bits we skipped.

Have fun! 💕